# Instruction Fine-tuning sobre un LLM Base

<div style="background-color:#D9EEFF;color:black;padding:2%;">
<h2>Fine Tuning Medical database</h2>

En este caso práctico, se proporne la realización de instruction fine-tuning sobre el LLM [Flan-T5-small](https://huggingface.co/google/flan-t5-small) con el objetivo de que sea capaz de iferir alguna enfermedad o afeccion dado unos sintomas.

</div>

# Resolución del caso práctico

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 0. Instalación de librerías externas

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=

## 1. Comportamiento de [Flan-T5-small](https://huggingface.co/google/flan-t5-small) sin Fine-tuning

### Lectura del modelo y tokenizador

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Importamos el tokenizador
tokenizer_FT5 = T5Tokenizer.from_pretrained("google/flan-t5-small")

# Importamos el modelo pre-entrenado
model_FT5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Lectura Dataset

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/drive/MyDrive/LLMs- Fine Tuning/cls_llm_dataset_final4.csv')

In [ ]:
data.head()

,Text,Label
0,Manchado o sangrado vaginal\r\nDolor o calambr...,Aborto espontáneo
1,"Dolor de muela intenso, persistente y grave qu...",Absceso dental
2,"Robar, falsificar o vender recetas\r\nTomar do...",Abuso de drogas recetadas
3,"Incapacidad para tragar (disfagia), que puede ...",Acalasia
4,Los cambios en la piel son los únicos signos d...,Acantosis pigmentaria


In [ ]:
n_register = np.random.randint(len(data))
text = data['Text'][n_register]
label = data['Label'][n_register]


print(f'\tNumero de registro: {n_register}')
print(f'*'*40)
print(f'\tTexto:\n{text}\n\n')
print(f'\t-> Label: {label}')


	Numero de registro: 248
****************************************
	Texto:
Desorientación
Dificultad para prestar atención y concentrarse
Capacidad reducida para organizar pensamientos o acciones
Deterioro en la capacidad para analizar una situación, desarrollar un plan eficaz y comunicar ese plan a otros
Lentitud al razonar
Dificultad con la organización
Dificultad para decidir qué hacer a continuación
Problemas con la memoria
Intranquilidad y agitación
Marcha inestable
Deseo repentino y frecuente de orinar o incapacidad para controlar la micción
Depresión o apatía
Los síntomas de la demencia vascular pueden ser más obvios cuando suceden de repente después de un accidente cerebrovascular. Cuando los cambios en el pensamiento y el razonamiento parecen estar claramente vinculados con un accidente cerebrovascular, en ocasiones esta afección se denomina demencia posterior a un accidente cerebrovascular.
A veces, un patrón característico de los síntomas de demencia vascular sigue una serie 

### Generación de texto

In [ ]:
context = data['Text'][0]

In [ ]:
context = "Tengo sangrado vaginal en el primer trimestre. ¿Qué enfermedad podría tener?"

In [ ]:
context = data['Text'][2]

In [ ]:
question = '¿Qué enfermedad o afección podría tener?'

In [ ]:
prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"

# Tokenizamos el prompt
prompt_tokens = tokenizer_FT5(prompt_template, return_tensors="pt").input_ids.to("cuda")

# Generamos los siguientes tokens
outputs = model_FT5.generate(prompt_tokens, max_length=200)

# Transformamos los tokens generados en texto
print(tokenizer_FT5.decode(outputs[0]))

<pad> enfermedad o afección</s>


## 2. Selección y preparación del conjunto de datos

### Formato del conjunto de datos

Es habitual utilizar plantillas que proponen los desarrolladores de los LLM para diseñar nuestros ejemplos de entrenamiento: https://github.com/google-research/FLAN/blob/main/flan/v2/flan_templates_branched.py

In [ ]:
# prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"
def convert_to_template(context, question):
    return f"Responde basado en contexto:\n\n{context}\n\n{question}"

In [ ]:
data['prompt'] = data.apply(lambda x: convert_to_template(x['Text'], question), axis=1)

In [ ]:
print(data['prompt'][0])

Responde basado en contexto:

Manchado o sangrado vaginal
Dolor o calambres en el abdomen o la parte inferior de la espalda
Fluidos o tejidos que salen por la vagina
Si ha salido tejido fetal por la vagina, colócalo en un recipiente limpio y llévalo al consultorio de tu profesional de salud o al hospital para que se lo analice.
Recuerda que la mayoría de las mujeres que presentan manchado o sangrado vaginal durante el primer trimestre continúan teniendo embarazos exitosos.

¿Qué enfermedad o afección podría tener?


In [ ]:
data.head()

,Text,Label,prompt
0,Manchado o sangrado vaginal\r\nDolor o calambr...,Aborto espontáneo,Responde basado en contexto:\n\nManchado o san...
1,"Dolor de muela intenso, persistente y grave qu...",Absceso dental,Responde basado en contexto:\n\nDolor de muela...
2,"Robar, falsificar o vender recetas\r\nTomar do...",Abuso de drogas recetadas,"Responde basado en contexto:\n\nRobar, falsifi..."
3,"Incapacidad para tragar (disfagia), que puede ...",Acalasia,Responde basado en contexto:\n\nIncapacidad pa...
4,Los cambios en la piel son los únicos signos d...,Acantosis pigmentaria,Responde basado en contexto:\n\nLos cambios en...


In [ ]:
data_cls = data[['prompt', 'Label']]
data_cls.head()

,prompt,Label
0,Responde basado en contexto:\n\nManchado o san...,Aborto espontáneo
1,Responde basado en contexto:\n\nDolor de muela...,Absceso dental
2,"Responde basado en contexto:\n\nRobar, falsifi...",Abuso de drogas recetadas
3,Responde basado en contexto:\n\nIncapacidad pa...,Acalasia
4,Responde basado en contexto:\n\nLos cambios en...,Acantosis pigmentaria


In [ ]:
# Dividimos el conjunto de datos en entrenamiento,test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(data_cls, test_size=0.2, random_state=42)

In [ ]:
# Convertimos a formato DatasetDict
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

ds = DatasetDict()

ds['train'] = train_dataset
ds['test'] = test_dataset

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 926
    })
    test: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 232
    })
})

In [ ]:
ds['train']['prompt'][1]

'Responde basado en contexto:\n\nProtuberancia en los puntos blandos (fontanelas) del cráneo del bebé\r\nNáuseas y vómitos\r\nRigidez corporal\r\nAlimentación deficiente o no despertarse para alimentarse\r\nIrritabilidad\r\nCuándo debes consultar a un médico\r\nObtén atención médica inmediata si tienes alguno de los síntomas más graves asociados con la encefalitis. Ante síntomas como dolor de cabeza intenso, fiebre y alteración del estado de conciencia, se requiere atención urgente.\r\nLos bebés y niños pequeños que presenten algún signo o síntoma de encefalitis deben recibir atención médica de urgencia.\n\n¿Qué enfermedad o afección podría tener?'

In [ ]:
ds['train']['Label'][1]

'Encefalitis'

In [ ]:
# Reducimos el conjunto de datos
NUM_EJ_TRAIN = 926
NUM_EJ_VAL = 232


# Subconjunto de entrenamiento
ds['train'] = ds['train'].select(range(NUM_EJ_TRAIN))

# Subconjunto de validación
ds['test'] = ds['test'].select(range(NUM_EJ_VAL))

In [ ]:
ds['test']['prompt'][1]

'Responde basado en contexto:\n\nHemorragia vaginal, aunque puede ser que no haya ninguna\r\nDolor abdominal\r\nDolor de espalda\r\nSensibilidad o rigidez uterina\r\nContracciones uterinas, a menudo una tras otra\r\nDolor abdominal y dolor de espalda que a menudo comienzan repentinamente. La cantidad de sangrado vaginal puede variar mucho y no necesariamente indica la cantidad de placenta que se ha separado del útero. Es posible que la sangre quede atrapada dentro del útero así que, incluso con un desprendimiento placentario grave, podría no haber sangrado visible.\r\nEn algunos casos, el desprendimiento placentario se desarrolla lentamente (desprendimiento crónico), lo que puede causar un ligero e intermitente sangrado vaginal. Es posible que el bebé no crezca tan rápido como se esperaba y que tenga un bajo nivel de líquido amniótico u otras complicaciones.\r\nCuándo consultar al médico\r\nBusca atención de emergencia si tienes signos o síntomas de desprendimiento de placenta.\n\n¿Qué

In [ ]:
ds['test']['Label'][1]

'Desprendimiento de placenta'

### 2.3. Tokenización del conjunto de datos

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [ ]:
from datasets import concatenate_datasets

# Calculamos el tamaño máximo de prompt
prompts_tokens = concatenate_datasets([ds["train"], ds["test"]]).map(lambda x: tokenizer(x["prompt"], truncation=True), batched=True)
max_token_len = max([len(x) for x in prompts_tokens["input_ids"]])
print(f"Maximo tamaño de prompt: {max_token_len}")

# Calculamos el tamaño máximo de completion
completions_tokens = concatenate_datasets([ds["train"], ds["test"]]).map(lambda x: tokenizer(x["Label"], truncation=True), batched=True)
max_completion_len = max([len(x) for x in completions_tokens["input_ids"]])
print(f"Maximo tamaño de completion: {max_completion_len}")

Map:   0%|          | 0/1158 [00:00<?, ? examples/s]

Maximo tamaño de prompt: 512


Map:   0%|          | 0/1158 [00:00<?, ? examples/s]

Maximo tamaño de completion: 38


In [ ]:
def padding_tokenizer(datos):
  # Tokenizar inputs (prompts)
  model_inputs = tokenizer(datos['prompt'], max_length=max_token_len, padding="max_length", truncation=True)

  # Tokenizar labels (completions)
  model_labels = tokenizer(datos['Label'], max_length=max_completion_len, padding="max_length", truncation=True)

  # Sustituimos el caracter de padding de las completion por -100 para que no se tenga en cuenta en el entrenamiento
  model_labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in model_labels["input_ids"]]

  model_inputs['labels'] = model_labels["input_ids"]

  return model_inputs

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 926
    })
    test: Dataset({
        features: ['prompt', 'Label', '__index_level_0__'],
        num_rows: 232
    })
})

In [ ]:
ds_tokens = ds.map(padding_tokenizer, batched=True , remove_columns=['prompt','Label'])
#remove_columns=['text', 'summary', 'topic', 'url', 'title', 'date', 'prompt']

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

In [ ]:
ds_tokens

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 926
    })
    test: Dataset({
        features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 232
    })
})

## 3. Fine-tuning del modelo

### 3.1. Lectura del modelo

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Cargamos el modelo
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

### 3.2. Evaluación durante el entrenamiento

A continuación implementamos un conjunto de funciones auxiliares para evluar los resultados durante el proceso de entrenamiento

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('punkt_tab')

# Metrica de evaluación
metric = evaluate.load("rouge")

# Funciona auxiliar para preprocesar el texto
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum espera una nueva línea después de cada frase
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Reemplazamos -100 en las etiquetas porque no podemos decodificarlo
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Preprocesamos el texto
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


### 3.3. Lectura y adaptación de los datos para el entrenamiento

In [ ]:
from transformers import DataCollatorForSeq2Seq

# Ignoramos los tokens relacionados con el padding durante el proceso de entrenamiento para los prompts
label_pad_token_id = -100

# Recolector de datos para el entrenamiento del modelo
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


### Preparación y ejecución del fine-tuning (entrenamiento)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["WANDB_DISABLED"] = "true"

REPOSITORY="/content/drive/MyDrive/LLMs- Fine Tuning/MedicalT5_Intruction_Fine_Tuning_LLM"

# Definimos las opciones del entrenamiento
training_args = Seq2SeqTrainingArguments(
    # Hiperprámetros del entrenamiento
    output_dir=REPOSITORY,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=45,
    # Estrategias de logging y evaluación
    logging_dir=f"{REPOSITORY}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Creamos la instancia de entrenamiento
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds_tokens["train"],
    eval_dataset=ds_tokens["test"],
    compute_metrics=compute_metrics,
    processing_class=tokenizer,  # <-- Aquí se usa processing_class en lugar de tokenizer
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Guardamos el tokenizador en disco para utilizarlo posteriormente
tokenizer.save_pretrained(f"{REPOSITORY}/Medical_tokenizer")

('/content/drive/MyDrive/LLMs- Fine Tuning/MedicalT5_Intruction_Fine_Tuning_LLM/Medical_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/LLMs- Fine Tuning/MedicalT5_Intruction_Fine_Tuning_LLM/Medical_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/LLMs- Fine Tuning/MedicalT5_Intruction_Fine_Tuning_LLM/Medical_tokenizer/spiece.model',
 '/content/drive/MyDrive/LLMs- Fine Tuning/MedicalT5_Intruction_Fine_Tuning_LLM/Medical_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/LLMs- Fine Tuning/MedicalT5_Intruction_Fine_Tuning_LLM/Medical_tokenizer/tokenizer.json')

In [ ]:

# Iniciamos el entrenamiento
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.981775,34.839900,20.425300,34.933100,34.848300,9.047414
2,No log,1.829099,38.481900,24.454800,38.529400,38.380300,9.974138
3,No log,1.772934,41.363600,26.480200,41.336400,41.309000,10.017241
4,No log,1.724838,43.038400,26.655300,43.114700,42.936400,10.646552
5,2.182100,1.704599,43.340600,26.933600,43.343100,43.273300,10.409483
6,2.182100,1.684887,45.008300,28.058000,45.166600,44.946300,10.650862
7,2.182100,1.666189,46.235100,28.222600,46.361000,46.206900,10.491379
8,2.182100,1.668435,45.703500,28.015700,45.902300,45.735100,10.383621
9,1.672300,1.657332,46.241700,28.574200,46.403000,46.320600,10.676724
10,1.672300,1.648902,45.619600,28.559000,45.743700,45.556800,10.469828


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=5220, training_loss=1.2361156039767796, metrics={'train_runtime': 2633.8646, 'train_samples_per_second': 15.821, 'train_steps_per_second': 1.982, 'total_flos': 7746057331015680.0, 'train_loss': 1.2361156039767796, 'epoch': 45.0})

## 4. Generación de texto con Flan-T5 Fine-tuned y evaluación

### Lectura del modelo y del tokenizador

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

REPOSITORY="/content/drive/MyDrive/LLMs- Fine Tuning/MedicalT5_Intruction_Fine_Tuning_LLM"

# Importamos el tokenizador
tokenizer_FT5_FT = T5Tokenizer.from_pretrained(f"{REPOSITORY}/Medical_tokenizer")

# Importamos el modelo con fine-tuning
model_FT5_FT = T5ForConditionalGeneration.from_pretrained(f"{REPOSITORY}/checkpoint-5220", device_map="auto")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### Generación de texto

In [ ]:
context = """
    Sudoración
    Escalofríos y temblores
    Dolor de cabeza
    Dolores musculares
    Pérdida de apetito
    Irritabilidad
    Deshidratación
    Debilidad general
"""
# Fiebre

In [8]:
context = """
Manchado o sangrado vaginal
Dolor o calambres en el abdomen o la parte inferior de la espalda
Fluidos o tejidos que salen por la vagina
Si ha salido tejido fetal por la vagina, colócalo en un recipiente limpio y llévalo al consultorio de tu profesional de salud o al hospital para que se lo analice.
Recuerda que la mayoría de las mujeres que presentan manchado o sangrado vaginal durante el primer trimestre continúan teniendo embarazos exitosos.
"""
# Aborto

In [9]:
context = """"
    Dolor de cabeza. \
    Tos persistente y seca. \
    Falta de aire. \
    Cansancio y debilidad. \
    Congestión o goteo nasal. \
    Dolor de garganta. \
    Dolor en los ojos. \
"""
# gripe(influencia)

In [ ]:
data

,Text,Label,prompt
0,Manchado o sangrado vaginal\r\nDolor o calambr...,Aborto espontáneo,Responde basado en contexto:\n\nManchado o san...
1,"Dolor de muela intenso, persistente y grave qu...",Absceso dental,Responde basado en contexto:\n\nDolor de muela...
2,"Robar, falsificar o vender recetas\r\nTomar do...",Abuso de drogas recetadas,"Responde basado en contexto:\n\nRobar, falsifi..."
3,"Incapacidad para tragar (disfagia), que puede ...",Acalasia,Responde basado en contexto:\n\nIncapacidad pa...
4,Los cambios en la piel son los únicos signos d...,Acantosis pigmentaria,Responde basado en contexto:\n\nLos cambios en...
...,...,...,...
1153,No se encontraron síntomas,Sarcoma de Kaposi,Responde basado en contexto:\n\nNo se encontra...
1154,No se encontraron síntomas,Sarcoma epitelioide,Responde basado en contexto:\n\nNo se encontra...
1155,No se encontraron síntomas,Sarcoma sinovial,Responde basado en contexto:\n\nNo se encontra...
1156,No se encontraron síntomas,Válvula aórtica bicúspide,Responde basado en contexto:\n\nNo se encontra...


In [ ]:
n_register = np.random.randint(len(data))
text = data['Text'][n_register]
label = data['Label'][n_register]


print(f'\tNumero de registro: {n_register}')
print(f'*'*40)
print(f'\tTexto:\n{text}\n\n')
print(f'\t-> Label: {label}')


	Numero de registro: 1126
****************************************
	Texto:
Los signos y síntomas del frenillo lingual corto incluyen los siguientes:
Dificultad para levantar la lengua hasta los dientes superiores o moverla de un lado a otro.
Problemas para sacar la lengua más allá de los dientes anteriores inferiores.
Una lengua que, cuando se saca, muestra una hendidura o tiene forma de corazón.
Cuándo debes consultar a un médico
Consulta con el médico si:
Tu bebé tiene signos de frenillo lingual corto que causan problemas, como al momento de amamantar.
Un patólogo del lenguaje cree que el habla de tu hijo se ve afectada por el frenillo lingual corto.
Tu hijo ya mayor se queja acerca de problemas que interfieren con su alimentación, habla o alcance de las muelas
Te molestan tus propios síntomas de frenillo lingual corto.
Por lo general, el frenillo de la lengua se separa antes del nacimiento para que la lengua tenga un rango de movimiento libre. Cuando el frenillo es corto, el frenill

In [ ]:
context = '''
Fiebre
Malestar general
Dolor de cabeza
Fatiga intensa
Dolor de espalda intenso
Vómitos, posiblemente
'''

In [11]:
question = '¿Qué enfermedad o afección podría tener?'

In [12]:
# Construimos el prompt conforme a la plantilla de fine-tuning
prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"

# Tokenizamos el prompt
prompt_tokens = tokenizer_FT5_FT(prompt_template, return_tensors="pt").input_ids.to("cuda")

# Generamos los siguientes tokens
outputs = model_FT5_FT.generate(prompt_tokens, max_length=300)

# Transformamos los tokens generados en texto
print(tokenizer_FT5_FT.decode(outputs[0]))

<pad> Cáncer de vejiga</s>


In [13]:
# Creamos una funcion para predecir un contexto
def predict_context(context, question):
    prompt_template = f"Responde basado en contexto:\n\n{context}\n\n{question}"

  # Tokenizamos el prompt
    prompt_tokens = tokenizer_FT5_FT(prompt_template, return_tensors="pt").input_ids.to("cuda")

    # Generamos los siguientes tokens
    outputs = model_FT5_FT.generate(prompt_tokens, max_length=300)

    # Transformamos los tokens generados en texto

    return tokenizer_FT5_FT.decode(outputs[0],skip_special_tokens=True)


Probando Algunas preguntas

In [ ]:
context = '''
No lograr aliviar los síntomas de la rinitis alérgica (fiebre del heno).
Los medicamentos para la alergia no me alivian o me causan efectos secundarios molestos.
Tengo otro trastorno que puede empeorar la rinitis alérgica, como pólipos nasales, asma o infecciones frecuentes de los senos paranasales.
'''
# Fiebre del heno
question = '¿Qué enfermedad o afección podría tener?'
respuesta =  predict_context(context,question)
print(respuesta)

Alergia en nios


In [ ]:
context = '''
Dolor en la parte baja del abdomen
Dolor al orinar
Necesidad de orinar a menudo
Dificultad para orinar o interrupción del flujo de orina
Sangre en la orina
Orina turbia o de un color oscuro anormal
'''
# Cálculos en la vejiga
question = '¿Qué enfermedad o afección podría tener?'
respuesta =  predict_context(context,question)
print(respuesta)

<pad> Cáncer de orina</s>


In [ ]:
context = '''
Dificultad repentina para respirar
Exceso de líquido en los pulmones (edema pulmonar)
Presión arterial baja repentina
Falla repentina del corazón para bombear sangre de manera efectiva (colapso cardiovascular)
Problemas potencialmente mortales con la coagulación de la sangre (coagulopatía intravascular diseminada)
Sangrado del útero, incisión por cesárea o lugares de vías intravenosas (IV)
Estado mental alterado, como ansiedad o sensación de fatalidad
Escalofríos
Frecuencia cardíaca acelerada o alteraciones en el ritmo de la frecuencia cardíaca
'''
# Embolia amniótica
question = '¿Qué enfermedad o afección podría tener?'
respuesta =  predict_context(context,question)
print(respuesta)

<pad> Cáncer de coagulación</s>


In [ ]:
context = '''
Amnesia disociativa. El síntoma principal es una pérdida de memoria que es más grave que un olvido normal y que no puede justificarse por la existencia de una enfermedad. No puedes recordar información sobre ti ni sobre acontecimientos y personas de tu vida, en especial los relacionados con un momento traumático. La amnesia disociativa puede ser específica de acontecimientos producidos en un cierto momento, como combates intensos, o, con menor frecuencia, puede tratarse de la pérdida completa de la memoria sobre ti mismo. A veces puede implicar que te traslades o deambules en un estado de confusión que te aleje de tu vida (fuga disociativa). El episodio de amnesia generalmente se presenta en forma repentina y puede durar minutos, horas o, rara vez, meses o años.
Amnesia disociativa.
Trastorno de identidad disociativo. Este trastorno, antes conocido como «trastorno de personalidad múltiple», se caracteriza por «alternar» diferentes identidades. Es posible que sientas la presencia de dos o más personas que hablan o viven en tu cabeza y que sientas que estas identidades te poseyeron. Cada identidad puede tener un nombre, una historia personal y características únicas, entre ellas, diferencias obvias de voz, género, tratos e incluso cualidades físicas, como la necesidad de usar lentes. También hay diferencias en cuanto a la familiaridad de cada identidad con las demás. Las personas con trastorno de identidad disociativo, en general, también tienen amnesia disociativa y, a menudo, sufren fuga disociativa.
Trastorno de identidad disociativo.
Trastorno de despersonalización-desrealización.
'''
question = '¿Qué enfermedad o afección podría tener?'
respuesta =  predict_context(context,question)
print(respuesta)

<pad> Trastorno de identidad disociativo</s>


In [ ]:
context = '''
Dolor en el pecho al respirar o toser
Desorientación o cambios de percepción mental (en adultos de 65 años o más)
Tos que puede producir flema
Fatiga
Fiebre, transpiración y escalofríos con temblor
Temperatura corporal más baja de lo normal (en adultos mayores de 65 años y personas con un sistema inmunitario débil)
Náuseas, vómitos o diarrea
Dificultad para respirar
'''
# Neumonia
question = '¿Qué enfermedad o afección podría tener?'
respuesta =  predict_context(context,question)
print(respuesta)

<pad> Fiebre</s>


### Evaluación con el subconjunto de pruebas

In [ ]:
import torch

# Cambiamos el modelo al modo de evaluación
model_FT5_FT.eval()

# Definir tamaño del lote
batch_size = 8

all_predictions = []

# Deshabilitamos el entrenamiento y obtenemos las completions
with torch.no_grad():
  for i in range(0, len(ds_tokens["test"]["input_ids"]), batch_size):
        # Extraemos el lote actual
        input_ids_batch = torch.tensor(ds_tokens["test"]["input_ids"][i:i+batch_size], device='cuda:0')

        # Obtenemos las predicciones del modelo
        outputs = model_FT5_FT.generate(input_ids_batch)

        # Concatenemos las predicciones
        all_predictions.extend(outputs)

# Calculamos las metricas
labels = np.array(ds_tokens['test']['labels'])

# Pad the predictions to the same length
max_len = max(len(pred) for pred in all_predictions)
padded_predictions = [np.pad(pred.cpu().numpy(), (0, max_len - len(pred)), 'constant', constant_values=tokenizer.pad_token_id) for pred in all_predictions]

completions = np.array(padded_predictions)

metrics = compute_metrics((completions, labels))

print(metrics)

{'rouge1': 44.1672, 'rouge2': 27.6365, 'rougeL': 44.3489, 'rougeLsum': 44.0882, 'gen_len': 10.392241379310345}


## Interfaz de prueba a usuario

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [14]:
#@title Asistente virtual
import ipywidgets as widgets
from IPython.display import display, HTML

# Aplicar estilo general al notebook
display(HTML("<style>.widget-label { font-size: 14px; font-weight: bold; }</style>"))

# Widget de área de texto para los síntomas
text_area = widgets.Textarea(
    value='Introduce aquí algunos síntomas . . .',
    placeholder='Ejemplo: Dolor de cabeza, fiebre, cansancio . . .',
    description='Síntomas:',
    layout=widgets.Layout(width='600px', height='150px', margin='10px 0'),
    style={'description_width': '100px'}
)

# Widget de salida estilizado
output_widget = widgets.Output(
    layout=widgets.Layout(
        border='1px solid #ccc',
        padding='10px',
        margin='10px 0',
        width='600px',
        height='auto'
    )
)

# Botón estilizado
submit_button = widgets.Button(
    description="Enviar",
    button_style='success',  # Cambia el estilo del botón ('success', 'info', 'warning', 'danger')
    icon='check',  # Icono de la librería FontAwesome
    tooltip='Haz clic para enviar'
)

# Función de manejo del botón
def handle_submit_button_click(b):
    sintomas = text_area.value.strip()
    if sintomas:  # Validación simple para evitar campos vacíos
        question = '¿Qué enfermedad o afección podría tener?'
        completion = predict_context(sintomas, question)  # Llamada a la función predict_context
        with output_widget:
            output_widget.clear_output()  # Limpiar la salida previa
            print(f"\nPosible enfermedad o afección:\n\n{completion}")
    else:
        with output_widget:
            output_widget.clear_output()
            print("\n⚠️ Por favor, introduce síntomas antes de enviar.")

# Asociar la función al evento click del botón
submit_button.on_click(handle_submit_button_click)

# Mostrar los widgets con diseño mejorado
display(widgets.VBox([text_area, submit_button, output_widget]))


In [19]:
#@title Obtener de registro

import numpy as np
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/LLMs- Fine Tuning/cls_llm_dataset_final4.csv')

def register_generator(data):
    n_register = np.random.randint(len(data))
    text = data['Text'][n_register]
    label = data['Label'][n_register]


    print(f'\tNumero de registro: {n_register}')
    print(f'*'*40)
    print(f'\tTexto:\n{text}\n\n')
    print(f'\t-> Label: {label}')

register_generator(data)


	Numero de registro: 846
****************************************
	Texto:
una sensación de ardor en el pecho que a veces se extiende hacia la garganta junto con un sabor amargo en la boca
Náuseas
Vómitos de un fluido amarillo verdoso (bilis)
Tos o ronquera ocasional
Pérdida de peso involuntaria
Cuándo debes consultar con un médico
Pide una cita con el médico si presentas síntomas de reflujo frecuentes, o si pierdes peso de forma no intencional.
Si te han diagnosticado enfermedad por reflujo gastroesofágico, pero no obtienes suficiente alivio con los medicamentos, llama al médico. Es posible que necesites otro tratamiento para el reflujo de bilis.


	-> Label: Reflujo biliar
